In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import sys
sys.path.append("/n/home12/binxuwang/Github/Closed-loop-visual-insilico")
import timm
import torch
import torch as th
import torch.nn as nn
from torchvision.models.feature_extraction import create_feature_extractor
from tqdm.auto import tqdm
from os.path import join
import pickle as pkl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from horama import maco, plot_maco
import torchvision.transforms as T
from torchvision.transforms import ToPILImage, ToTensor, Normalize, Resize
from torchvision.models import resnet50
from circuit_toolkit.CNN_scorers import TorchScorer
from circuit_toolkit.GAN_utils import upconvGAN, Caffenet
from circuit_toolkit.plot_utils import to_imgrid, show_imgrid, save_imgrid, saveallforms
from circuit_toolkit.layer_hook_utils import featureFetcher_module, featureFetcher, get_module_names
from circuit_toolkit.dataset_utils import ImagePathDataset
from torch.utils.data import DataLoader
from neural_regress.regress_lib import sweep_regressors
from neural_regress.sklearn_torchify_lib import SRP_torch, PCA_torch, LinearRegression_torch, SpatialAvg_torch

import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.decomposition import PCA
from sklearn.kernel_ridge import KernelRidge

#%% Utility Functions
from core.data_utils import load_from_hdf5, load_neural_data, load_neural_trial_resp_tensor, create_response_tensor, parse_image_fullpaths

In [8]:
from core.data_utils import load_neural_data, load_from_hdf5, load_neural_trial_resp_tensor, create_response_tensor
dataroot = r"/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Ephys_Data"
data_path = join(dataroot, "red_20250428-20240429_vvs-encodingstimuli_z1_rw100-400.h5")
data = load_from_hdf5(data_path)

In [9]:
encoding_stim_dir = r"/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Stimuli/encodingstimuli_apr2025"
data_path = join(dataroot, "red_20250428-20240429_vvs-encodingstimuli_z1_rw100-400.h5")
data = load_from_hdf5(data_path)

subject_id = "red_20250428-20240429"

data_dict = {}
data_dict['image_fps'] = parse_image_fullpaths(data['repavg']["stimulus_name"], [encoding_stim_dir], arbitrary_format=True)
data_dict['resp_mat'] = data['repavg']["response_peak"]
data_dict['resp_temp_mat'] = data['repavg']["response_temporal"]
data_dict['reliability'] = data['neuron_metadata']["reliability"]
data_dict['ncsnr'] = data['neuron_metadata']["ncsnr"]
data_dict['brain_area'] = data['neuron_metadata']["brain_area"]
data_dict['stim_pos'] = data['stimulus_meta']['xy_deg']
data_dict['stim_size'] = data["stimulus_meta"]["size_px"]

image_fps = data_dict['image_fps']
resp_mat = data_dict['resp_mat']
reliability = data_dict['reliability']
ncsnr = data_dict['ncsnr']

All stimulus files were found


In [10]:
import sys
sys.path.append("/n/home12/binxuwang/Github/Closed-loop-visual-insilico")
from core.model_load_utils import load_model_transform, MODEL_LAYER_FILTERS, LAYER_ABBREVIATION_MAPS
from neural_regress.regress_lib import record_features, perform_regression_sweeplayer, perform_regression_sweeplayer_RidgeCV
from neural_regress.regress_lib import sweep_regressors, transform_features2Xdict, RidgeCV
from neural_regress.regress_eval_lib import format_result_df, plot_result_df_per_layer, construct_result_df_masked, \
    compute_pred_dict_D2_per_unit

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
batch_size = 96

model_names = [
    "resnet50_robust",
    "resnet50",
    "resnet50_clip",
    "resnet50_dino",
    "clipag_vitb32",
    "siglip2_vitb16",
    "dinov2_vitb14_reg",
    "radio_v2.5-b",
]

for modelname in model_names:
    print(f"\nTesting model: {modelname}")
    try:
        # Load model and transforms
        model, transforms_pipeline = load_model_transform(modelname, device)
        print(f"Successfully loaded model: {modelname}")
        dataset = ImagePathDataset(image_fps, scores=None, transform=transforms_pipeline)
        print(f"Successfully created dataset for {modelname} with {len(dataset)} images")
        # Create feature fetcher
        fetcher = featureFetcher(model, input_size=(3, 224, 224), print_module=False)
        print(f"Successfully created feature fetcher for {modelname}")
        # Get all module names
        all_module_names = list(fetcher.module_names.values())
        print(f"Total number of modules: {len(all_module_names)}")
        # Get layer filter for this model
        layer_filter = MODEL_LAYER_FILTERS[modelname]
        layer_abbrev = LAYER_ABBREVIATION_MAPS[modelname]
        # Count layers that pass the filter
        module_names2record = [name for name in all_module_names if layer_filter(name)]
        print(f"Number of layers passing filter: {len(module_names2record)}")
        print(f"Filtered layer names: {module_names2record}")
        module_names2record_abbrev = [layer_abbrev(name) for name in module_names2record]
        print(f"Filtered layer names (abbreviated): {module_names2record_abbrev}")
    except Exception as e:
        print(f"Error loading model {modelname}: {str(e)}")
    
    # # hook the layers
    # for name in module_names2record: 
    #     fetcher.record(name, store_device='cpu', ingraph=False, )
    
    # # Record features
    # feat_dict_lyrswp = record_features(model, fetcher, dataset, batch_size=batch_size, device=device)
    # # Cleanup
    # print(f"{modelname} done!!!")
    # fetcher.cleanup()x
    


Using device: cuda

Testing model: resnet50_robust


/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Successfully loaded model: resnet50_robust
The (227, 227) setting is overwritten by the size in custom transform
Successfully created dataset for resnet50_robust with 969 images
Successfully created feature fetcher for resnet50_robust
Total number of modules: 184
Number of layers passing filter: 16
Filtered layer names: ['.layer1.Bottleneck0', '.layer1.Bottleneck1', '.layer1.Bottleneck2', '.layer2.Bottleneck0', '.layer2.Bottleneck1', '.layer2.Bottleneck2', '.layer2.Bottleneck3', '.layer3.Bottleneck0', '.layer3.Bottleneck1', '.layer3.Bottleneck2', '.layer3.Bottleneck3', '.layer3.Bottleneck4', '.layer3.Bottleneck5', '.layer4.Bottleneck0', '.layer4.Bottleneck1', '.layer4.Bottleneck2']
Filtered layer names (abbreviated): ['L1.B0', 'L1.B1', 'L1.B2', 'L2.B0', 'L2.B1', 'L2.B2', 'L2.B3', 'L3.B0', 'L3.B1', 'L3.B2', 'L3.B3', 'L3.B4', 'L3.B5', 'L4.B0', 'L4.B1', 'L4.B2']

Testing model: resnet50


/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Successfully loaded model: resnet50
The (227, 227) setting is overwritten by the size in custom transform
Successfully created dataset for resnet50 with 969 images
Successfully created feature fetcher for resnet50
Total number of modules: 184
Number of layers passing filter: 16
Filtered layer names: ['.layer1.Bottleneck0', '.layer1.Bottleneck1', '.layer1.Bottleneck2', '.layer2.Bottleneck0', '.layer2.Bottleneck1', '.layer2.Bottleneck2', '.layer2.Bottleneck3', '.layer3.Bottleneck0', '.layer3.Bottleneck1', '.layer3.Bottleneck2', '.layer3.Bottleneck3', '.layer3.Bottleneck4', '.layer3.Bottleneck5', '.layer4.Bottleneck0', '.layer4.Bottleneck1', '.layer4.Bottleneck2']
Filtered layer names (abbreviated): ['L1.B0', 'L1.B1', 'L1.B2', 'L2.B0', 'L2.B1', 'L2.B2', 'L2.B3', 'L3.B0', 'L3.B1', 'L3.B2', 'L3.B3', 'L3.B4', 'L3.B5', 'L4.B0', 'L4.B1', 'L4.B2']

Testing model: resnet50_clip
Successfully loaded model: resnet50_clip
The (227, 227) setting is overwritten by the size in custom transform
Successf

...

Successfully loaded model: clipag_vitb32
The (227, 227) setting is overwritten by the size in custom transform
Successfully created dataset for clipag_vitb32 with 969 images
Successfully created feature fetcher for clipag_vitb32
Total number of modules: 127
Number of layers passing filter: 12
Filtered layer names: ['.transformer.resblocks.ResidualAttentionBlock0', '.transformer.resblocks.ResidualAttentionBlock1', '.transformer.resblocks.ResidualAttentionBlock2', '.transformer.resblocks.ResidualAttentionBlock3', '.transformer.resblocks.ResidualAttentionBlock4', '.transformer.resblocks.ResidualAttentionBlock5', '.transformer.resblocks.ResidualAttentionBlock6', '.transformer.resblocks.ResidualAttentionBlock7', '.transformer.resblocks.ResidualAttentionBlock8', '.transformer.resblocks.ResidualAttentionBlock9', '.transformer.resblocks.ResidualAttentionBlock10', '.transformer.resblocks.ResidualAttentionBlock11']
Filtered layer names (abbreviated): ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B

Using cache found in /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/torch_cache/hub/facebookresearch_dinov2_main
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/torch_cache/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/torch_cache/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/torch_cache/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


Successfully loaded model: dinov2_vitb14_reg
The (227, 227) setting is overwritten by the size in custom transform
Successfully created dataset for dinov2_vitb14_reg with 969 images
Successfully created feature fetcher for dinov2_vitb14_reg
Total number of modules: 187
Number of layers passing filter: 12
Filtered layer names: ['.blocks.NestedTensorBlock0', '.blocks.NestedTensorBlock1', '.blocks.NestedTensorBlock2', '.blocks.NestedTensorBlock3', '.blocks.NestedTensorBlock4', '.blocks.NestedTensorBlock5', '.blocks.NestedTensorBlock6', '.blocks.NestedTensorBlock7', '.blocks.NestedTensorBlock8', '.blocks.NestedTensorBlock9', '.blocks.NestedTensorBlock10', '.blocks.NestedTensorBlock11']
Filtered layer names (abbreviated): ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11']

Testing model: radio_v2.5-b


Using cache found in /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/torch_cache/hub/NVlabs_RADIO_main


Successfully loaded model: radio_v2.5-b
The (227, 227) setting is overwritten by the size in custom transform
Successfully created dataset for radio_v2.5-b with 969 images
Successfully created feature fetcher for radio_v2.5-b
Total number of modules: 251
Number of layers passing filter: 12
Filtered layer names: ['.model.blocks.Block0', '.model.blocks.Block1', '.model.blocks.Block2', '.model.blocks.Block3', '.model.blocks.Block4', '.model.blocks.Block5', '.model.blocks.Block6', '.model.blocks.Block7', '.model.blocks.Block8', '.model.blocks.Block9', '.model.blocks.Block10', '.model.blocks.Block11']
Filtered layer names (abbreviated): ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11']


In [ ]:
!ls /n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Encoding_models

Encoding_model_outputs	Ephys_Data  Reduced_Data  Stimuli


In [11]:
df_stim = pd.read_csv(join(encoding_stim_dir, "encoding_stimuli_split_seed0.csv"), )
train_idx = df_stim[df_stim["is_train"]].index

In [12]:
train_idx

Index([  0,   1,   2,   3,   4,   5,   7,   8,  10,  11,
       ...
       957, 958, 959, 960, 961, 962, 964, 965, 966, 968],
      dtype='int64', length=774)

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
batch_size = 96

model_names = [
    "resnet50_robust",
    # "resnet50",
    # "resnet50_clip",
    # "resnet50_dino",
    # "clipag_vitb32",
    # "siglip2_vitb16",
    # "dinov2_vitb14_reg",
    # "radio_v2.5-b",
]

outputroot = r"/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Encoding_models"
output_name = "model_outputs"
figdir = join(outputroot, subject_id, output_name)
os.makedirs(figdir, exist_ok=True)
for modelname in model_names:
    print(f"\nTesting model: {modelname}")
    try:
        # Load model and transforms
        model, transforms_pipeline = load_model_transform(modelname, device)
        print(f"Successfully loaded model: {modelname}")
        dataset = ImagePathDataset(image_fps, scores=None, transform=transforms_pipeline)
        print(f"Successfully created dataset for {modelname} with {len(dataset)} images")
        # Create feature fetcher
        fetcher = featureFetcher(model, input_size=(3, 224, 224), print_module=False)
        print(f"Successfully created feature fetcher for {modelname}")
        # Get all module names
        all_module_names = list(fetcher.module_names.values())
        print(f"Total number of modules: {len(all_module_names)}")
        # Get layer filter for this model
        layer_filter = MODEL_LAYER_FILTERS[modelname]
        layer_abbrev = LAYER_ABBREVIATION_MAPS[modelname]
        # Count layers that pass the filter
        module_names2record = [name for name in all_module_names if layer_filter(name)]
        print(f"Number of layers passing filter: {len(module_names2record)}")
        print(f"Filtered layer names: {module_names2record}")
        module_names2record_abbrev = [layer_abbrev(name) for name in module_names2record]
        print(f"Filtered layer names (abbreviated): {module_names2record_abbrev}")
    except Exception as e:
        print(f"Error loading model {modelname}: {str(e)}")
    
    # # hook the layers
    for name in module_names2record: 
        fetcher.record(name, store_device='cpu', ingraph=False, )
    
    # Record features
    feat_dict_lyrswp = record_features(model, fetcher, dataset, batch_size=batch_size, device=device)
    # Cleanup
    print(f"{modelname} done!!!")
    
    fetcher.cleanup()
    resp_mat_sel = resp_mat[:, :] # Select all channels, no mask
    print(f"Fitting models for All channels N={resp_mat_sel.shape[1]}")
    Xdict_lyrswp, Xtfmer_lyrswp = transform_features2Xdict(feat_dict_lyrswp, module_names2record, 
                            dimred_list=["pca750", "srp", ],  #  "srp"
                            pretrained_Xtransforms={}, use_pca_dual=True, use_srp_torch=True, train_split_idx=train_idx)
    regressors = [RidgeCV(alphas=[1E-4, 1E-3, 1E-2, 1E-1, 1, 10, 100, 1E3, 1E4, 1E5, 1E6, 1E7, 1E8, 1E9], 
                        alpha_per_target=True,),
                # MultiTaskLassoCV(cv=5, n_alphas=100, n_jobs=-1, max_iter=10000, tol=1E-4), 
                # MultiOutputSeparateLassoCV(cv=5, n_alphas=100, n_jobs=-1, max_iter=10000, tol=1E-4), 
                ] 
    regressor_names = ["RidgeCV"]
    result_df_lyrswp, fit_models_lyrswp = sweep_regressors(Xdict_lyrswp, resp_mat_sel, regressors, regressor_names, 
                                                        verbose=True)
    pred_D2_dict = compute_pred_dict_D2_per_unit(fit_models_lyrswp, Xdict_lyrswp, resp_mat_sel)
    pkl.dump(pred_D2_dict, open(join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_pred_meta.pkl"), "wb"))
    result_df_lyrswp.to_csv(join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_sweep_RidgeCV.csv"))
    th.save(fit_models_lyrswp, join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_fitmodels_RidgeCV.pth")) 
    # th.save(Xtfmer_lyrswp, join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_Xtfmer_RidgeCV.pth"))
    Xtfmer_lyrswp_sparse = {k: v for k, v in Xtfmer_lyrswp.items() if "layer4" in k and "pca" in k}
    th.save(Xtfmer_lyrswp_sparse, join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_Xtfmer_RidgeCV.pth"))
    # pkl.dump(Xtfmer_lyrswp, open(join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_Xtfmer_RidgeCV.pkl"), "wb"))
    # %%
    figh = plot_result_df_per_layer(result_df_lyrswp, )
    figh.suptitle(f"{subject_id} {modelname} layer sweep")
    figh.tight_layout()
    saveallforms(figdir, f"{subject_id}_{modelname}_layer_sweep_synopisis", figh=figh)
    # %%
    # Mask out unreliable channels and plot again
    for thresh in [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]:
        channel_count = (reliability > thresh).sum()
        result_df_masked = construct_result_df_masked(pred_D2_dict['D2_per_unit_train_dict'], 
                                                    pred_D2_dict['D2_per_unit_test_dict'], 
                                                    mask=reliability > thresh)
        figh = plot_result_df_per_layer(result_df_masked, )
        figh.suptitle(f"{subject_id} {modelname} layer sweep | reliable channels > {thresh} (N={channel_count})")
        figh.tight_layout()
        figh.show()
        saveallforms(figdir, f"{subject_id}_{modelname}_layer_sweep_synopisis_reliable_thresh{thresh}_masked", figh=figh)
    plt.close("all")
    


Using device: cuda

Testing model: resnet50_robust


/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Successfully loaded model: resnet50_robust
The (227, 227) setting is overwritten by the size in custom transform
Successfully created dataset for resnet50_robust with 969 images
Successfully created feature fetcher for resnet50_robust
Total number of modules: 184
Number of layers passing filter: 16
Filtered layer names: ['.layer1.Bottleneck0', '.layer1.Bottleneck1', '.layer1.Bottleneck2', '.layer2.Bottleneck0', '.layer2.Bottleneck1', '.layer2.Bottleneck2', '.layer2.Bottleneck3', '.layer3.Bottleneck0', '.layer3.Bottleneck1', '.layer3.Bottleneck2', '.layer3.Bottleneck3', '.layer3.Bottleneck4', '.layer3.Bottleneck5', '.layer4.Bottleneck0', '.layer4.Bottleneck1', '.layer4.Bottleneck2']
Filtered layer names (abbreviated): ['L1.B0', 'L1.B1', 'L1.B2', 'L2.B0', 'L2.B1', 'L2.B2', 'L2.B3', 'L3.B0', 'L3.B1', 'L3.B2', 'L3.B3', 'L3.B4', 'L3.B5', 'L4.B0', 'L4.B1', 'L4.B2']


  0%|          | 0/11 [00:00<?, ?it/s]

.layer1.Bottleneck0 torch.Size([969, 256, 56, 56])
.layer1.Bottleneck1 torch.Size([969, 256, 56, 56])
.layer1.Bottleneck2 torch.Size([969, 256, 56, 56])
.layer2.Bottleneck0 torch.Size([969, 512, 28, 28])
.layer2.Bottleneck1 torch.Size([969, 512, 28, 28])
.layer2.Bottleneck2 torch.Size([969, 512, 28, 28])
.layer2.Bottleneck3 torch.Size([969, 512, 28, 28])
.layer3.Bottleneck0 torch.Size([969, 1024, 14, 14])
.layer3.Bottleneck1 torch.Size([969, 1024, 14, 14])
.layer3.Bottleneck2 torch.Size([969, 1024, 14, 14])
.layer3.Bottleneck3 torch.Size([969, 1024, 14, 14])
.layer3.Bottleneck4 torch.Size([969, 1024, 14, 14])
.layer3.Bottleneck5 torch.Size([969, 1024, 14, 14])
.layer4.Bottleneck0 torch.Size([969, 2048, 7, 7])
.layer4.Bottleneck1 torch.Size([969, 2048, 7, 7])
.layer4.Bottleneck2 torch.Size([969, 2048, 7, 7])
resnet50_robust done!!!
FeatureFetcher hooks all freed
Fitting models for All channels N=64
.layer1.Bottleneck0 torch.Size([969, 256, 56, 56])
Time taken to transform .layer1.Bottle

/n/home12/binxuwang/Github/Closed-loop-visual-insilico/neural_regress/regress_eval_lib.py:125: RuntimeWarning: Mean of empty slice.
  {"train_score": D2_per_unit_train_masked.mean(), "test_score": D2_per_unit_test_masked.mean(), }
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [14]:
list(Xtfmer_lyrswp)

['.layer1.Bottleneck0_pca750',
 '.layer1.Bottleneck0_srp',
 '.layer1.Bottleneck1_pca750',
 '.layer1.Bottleneck1_srp',
 '.layer1.Bottleneck2_pca750',
 '.layer1.Bottleneck2_srp',
 '.layer2.Bottleneck0_pca750',
 '.layer2.Bottleneck0_srp',
 '.layer2.Bottleneck1_pca750',
 '.layer2.Bottleneck1_srp',
 '.layer2.Bottleneck2_pca750',
 '.layer2.Bottleneck2_srp',
 '.layer2.Bottleneck3_pca750',
 '.layer2.Bottleneck3_srp',
 '.layer3.Bottleneck0_pca750',
 '.layer3.Bottleneck0_srp',
 '.layer3.Bottleneck1_pca750',
 '.layer3.Bottleneck1_srp',
 '.layer3.Bottleneck2_pca750',
 '.layer3.Bottleneck2_srp',
 '.layer3.Bottleneck3_pca750',
 '.layer3.Bottleneck3_srp',
 '.layer3.Bottleneck4_pca750',
 '.layer3.Bottleneck4_srp',
 '.layer3.Bottleneck5_pca750',
 '.layer3.Bottleneck5_srp',
 '.layer4.Bottleneck0_pca750',
 '.layer4.Bottleneck0_srp',
 '.layer4.Bottleneck1_pca750',
 '.layer4.Bottleneck1_srp',
 '.layer4.Bottleneck2_pca750',
 '.layer4.Bottleneck2_srp']

In [15]:
Xtfmer_lyrswp_sparse = {k: v for k, v in Xtfmer_lyrswp.items() if "layer4" in k and "pca" in k}
th.save(Xtfmer_lyrswp_sparse, join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_Xtfmer_RidgeCV.pth"))

In [17]:
pkl.dump(Xtfmer_lyrswp, open(join(figdir, f"{subject_id}_{modelname}_sweep_regressors_layers_Xtfmer_RidgeCV.pkl"), "wb"))